In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import regions
from astropy.table import *
from tqdm import tqdm
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.wcs import WCS

In [2]:
import matplotlib

plt.rc('font', family='serif'), plt.rc('xtick', labelsize=18), plt.rc('ytick', labelsize=18)
plt.rcParams['savefig.dpi'] = 300
plt.rc('text',usetex=True)
matplotlib.rcParams['xtick.direction'] = 'in'
matplotlib.rcParams['ytick.direction'] = 'in'
np.set_printoptions(precision=3)

### load tables

In [4]:
cat = Table.read('/Volumes/LejaySSD/SHIRAZ_catalogs/DEEP23_CH1_newfluxes_errmod.cat',format='ascii')

In [5]:
cat.info

<Table length=434226>
     name       dtype 
-------------- -------
        NUMBER   int64
     FLUX_APER float64
   FLUX_APER_1 float64
   FLUX_APER_2 float64
   FLUX_APER_3 float64
   FLUX_APER_4 float64
   FLUX_APER_5 float64
   FLUX_APER_6 float64
   FLUX_APER_7 float64
  FLUXERR_APER float64
FLUXERR_APER_1 float64
FLUXERR_APER_2 float64
FLUXERR_APER_3 float64
FLUXERR_APER_4 float64
FLUXERR_APER_5 float64
FLUXERR_APER_6 float64
FLUXERR_APER_7 float64
      MAG_APER float64
    MAG_APER_1 float64
    MAG_APER_2 float64
    MAG_APER_3 float64
    MAG_APER_4 float64
    MAG_APER_5 float64
    MAG_APER_6 float64
    MAG_APER_7 float64
   MAGERR_APER float64
 MAGERR_APER_1 float64
 MAGERR_APER_2 float64
 MAGERR_APER_3 float64
 MAGERR_APER_4 float64
 MAGERR_APER_5 float64
 MAGERR_APER_6 float64
 MAGERR_APER_7 float64
     FLUX_AUTO float64
  FLUXERR_AUTO float64
      MAG_AUTO float64
   MAGERR_AUTO float64
   KRON_RADIUS float64
       X_IMAGE float64
       Y_IMAGE float64
   ALPHA_J20

### match with Vincent's catalog

In [ ]:
cat_shiraz_name =
cat_v8_name = 

cmd = ''
print(cmd)

### statistics

### SF/Q plots